In [1]:
from pstats import SortKey
import cProfile, pstats, yaml, os
from llama_index import SimpleDirectoryReader

In [2]:
# !llamaindex-cli download-llamadataset PatronusAIFinanceBenchDataset --download-dir ./data

In [3]:
reader = SimpleDirectoryReader(input_dir="./data")

## Sequential Load

In [4]:
documents = reader.load_data()
len(documents)

4478

In [5]:
cProfile.run("reader.load_data()", "stats/oldstats")
p = pstats.Stats("stats/oldstats")
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(15)

Mon Jan 15 14:34:15 2024    stats/oldstats

         2071618291 function calls (2067598183 primitive calls) in 362.165 seconds

   Ordered by: cumulative time
   List reduced from 310 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  362.165  362.165 {built-in method builtins.exec}
        1    0.002    0.002  362.165  362.165 <string>:1(<module>)
        1    0.000    0.000  362.163  362.163 base.py:343(load_data)
       33    0.000    0.000  362.161   10.975 base.py:255(load_file)
       33    0.027    0.001  362.156   10.974 docs_reader.py:24(load_data)
     4478    1.126    0.000  358.765    0.080 _page.py:2190(extract_text)
4789/4478    5.701    0.001  357.639    0.080 _page.py:1856(_extract_text)
     4789    0.006    0.000  268.299    0.056 _data_structures.py:1238(operations)
     4618   53.654    0.012  268.292    0.058 _data_structures.py:1102(_parse_content_stream)
42389034/39086149   30.553   

# Parallel Load

In [6]:
documents = reader.load_data(num_workers=10)
len(documents)

4478

In [7]:
cProfile.run("reader.load_data(num_workers=10)", "stats/newstats")
p = pstats.Stats("stats/newstats")
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(15)

Mon Jan 15 14:35:33 2024    stats/newstats

         13085 function calls in 38.536 seconds

   Ordered by: cumulative time
   List reduced from 213 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   38.536   38.536 {built-in method builtins.exec}
        1    0.004    0.004   38.536   38.536 <string>:1(<module>)
        1    0.000    0.000   38.532   38.532 base.py:343(load_data)
       24    0.000    0.000   38.447    1.602 threading.py:563(wait)
       24    0.000    0.000   38.447    1.602 threading.py:280(wait)
      159   38.447    0.242   38.447    0.242 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000   38.438   38.438 pool.py:366(starmap)
        1    0.000    0.000   38.438   38.438 pool.py:764(get)
        1    0.000    0.000   38.438   38.438 pool.py:761(wait)
        1    0.000    0.000    0.070    0.070 context.py:115(Pool)
        1    0.000    0.000    0.069    0.06

## Conclusion

In [8]:
SeqentialProcessingTime = 360.101
ParallelProcessingTime = 38.349

SpeedUp = SeqentialProcessingTime / ParallelProcessingTime
SpeedUp = round(SpeedUp * 100, 2)
print(f"SpeedUp: {SpeedUp}%")

SpeedUp: 939.01%
